Для выполнения задачи по подсчету людей в видеопоследовательностях с использованием трекера YOLO, детектора YOLO от Ultralytics, и реид-сети OSNet, нужно выполнить следующие шаги:

1. **Загрузка и подготовка данных**: Скачаны видеопоследовательности в виде изображений и преобразованы в видеофайлы
2. **Детекция и трекинг**: Настроены детекции людей с помощью YOLOv8n и трекинг людей с помощью BoxMOT методом DeepOCSORT
3. **ReID и улучшение трекинга**: Использована ReID сеть от YOLO (boxmot) для улучшения трекинга и предотвращения перепутывания идентификаторов
4. **Подсчет людей**: Реализована логика подсчета уникальных людей
5. **Визуализация**: Показаны треки и bounding boxes с подсчетом на видео

## Установка зависимостей

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from moviepy.editor import *
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
!rm -rf boxmot
!git clone https://github.com/mikel-brostrom/boxmot.git

Cloning into 'boxmot'...
remote: Enumerating objects: 15135, done.
remote: Counting objects: 100% (3056/3056), done.
remote: Compressing objects: 100% (997/997), done.
remote: Total 15135 (delta 2203), reused 2797 (delta 2047), pack-reused 12079 (from 1)
Receiving objects: 100% (15135/15135), 112.08 MiB | 24.33 MiB/s, done.
Resolving deltas: 100% (9719/9719), done.


In [ ]:
!pip install boxmot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
# ! git clone https://github.com/mikel-brostrom/yolo_tracking.git±
! cd boxmot && pip install poetry && poetry install --with yolo  && poetry shell
# ! cd yolo_tracking && poetry install --with yolo && poetry shell

Выходные данные были обрезаны до нескольких последних строк (5000).
  - Installing nvidia-cusolver-cu12 (11.4.5.107): Downloading... 79%
  - Installing nvidia-nccl-cu12 (2.19.3): Downloading... 48%
  - Installing nvidia-nvtx-cu12 (12.1.105)
  - Installing packaging (24.1)
  - Installing pillow (10.4.0)
  - Installing pyparsing (3.1.4)
  - Installing python-dateutil (2.9.0.post0)
  - Installing pytz (2024.2)
  - Installing referencing (0.35.1)
  - Installing sympy (1.13.2): Installing...
  - Installing nvidia-nvtx-cu12 (12.1.105)
  - Installing packaging (24.1)
  - Installing pillow (10.4.0)
  - Installing pyparsing (3.1.4)
  - Installing python-dateutil (2.9.0.post0)
  - Installing pytz (2024.2)
  - Installing referencing (0.35.1)
  - Installing sympy (1.13.2): Installing...
  - Installing nvidia-nccl-cu12 (2.19.3): Downloading... 50%
  - Installing nvidia-nvtx-cu12 (12.1.105)
  - Installing packaging (24.1)
  - Installing pillow (10.4.0)
  - Installing pyparsing (3.1.4)
  - Installing

In [ ]:
%pip install ultralytics opencv-python-headless opencv-contrib-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.3/881.3 kB 21.7 MB/s eta 0:00:00


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.1 🚀 Python-3.10.12 torch-2.2.2+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 55.5/107.7 GB disk)


In [ ]:
import cv2
print(cv2.__version__)

4.10.0


## Сборка видеофайлов

In [ ]:
import os

In [ ]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
!ffmpeg -pattern_type glob -framerate 30 -f image2 -i '/content/drive/MyDrive/datasets/people_counting_otus/1/*.jpg' video1.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!ffmpeg -pattern_type glob -framerate 30 -f image2 -i '/content/drive/MyDrive/datasets/people_counting_otus/3/*.jpg' video3.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!ffmpeg -pattern_type glob -framerate 30 -f image2 -i '/content/drive/MyDrive/datasets/people_counting_otus/96/*.jpg' video96.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## Настройка YOLOv8 для детекции людей

In [ ]:
from ultralytics import YOLO

# Инициализация модели YOLOv8 для детекции
model = YOLO('yolov8n.pt')

# Функция детекции
def detect_people(frame):
    results = model(frame)
    dets = []
    for result in results[0].boxes:
        class_id = int(result.cls[0].cpu().numpy())
        # Фильтруем только объекты класса 'person', у которых class_id == 0
        if class_id == 0:
            x1, y1, x2, y2 = result.xyxy[0].cpu().numpy()
            score = result.conf[0].cpu().numpy()
            dets.append([x1, y1, x2, y2, score, class_id])
    return dets


100%|██████████| 6.25M/6.25M [00:00<00:00, 95.1MB/s]


## Настройка трекера

Мы можем использовать трекер YOLO-tracking или другой трекер, который поддерживает работу с результатами детекции.

Пример использования YOLO-tracking с возможностью интеграции с YOLOv8:

In [ ]:
!pip show boxmot

Name: boxmot
Version: 11.0.2
Summary: BoxMOT: pluggable SOTA tracking modules for segmentation, object detection and pose estimation models
Home-page: 
Author: Mikel Broström
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: filterpy, ftfy, gdown, gitpython, lapx, loguru, numpy, opencv-python, pandas, pyyaml, regex, scikit-learn, torch, torchvision, yacs
Required-by: 


In [ ]:
from boxmot import DeepOCSORT

# Инициализация трекера
tracker = DeepOCSORT(
    reid_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device='cpu',
    half=False,
    iou_threshold=0.5
)

# Функция трекинга
def track_people(dets, im):
    # Передаем детекции трекеру
    dets_np = np.array(dets)
    tracked_objects = tracker.update(dets_np, im)
    return tracked_objects


  warnings.warn(

2024-09-30 20:44:56.975 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v11.0.1 🚀 Python-3.10.12 torch-2.2.2+cu121CPU
Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: /content/osnet_x0_25_msmt17.pt
100%|██████████| 3.06M/3.06M [00:00<00:00, 21.9MB/s]
2024-09-30 20:45:01.162 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


## Подсчет уникальных людей

Логика подсчета может основываться на уникальных ID от трекера:

In [ ]:
def count_unique_people(tracked_objects, path):
    global tracked_ids
    for track in tracked_objects:
        track_id = track[7]
        tracked_ids.add(track_id)

## Визуализация треков и bounding boxes


In [ ]:
def visualize_tracks(tracks, frame):
    for track in tracks:
        x1, y1, x2, y2 = map(int, track[:4])  # Координаты bounding box
        track_id = int(track[7])  # Track ID

        # Отрисовка bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Отображение track_id на bounding box
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return frame

## Основной цикл обработки видео

In [ ]:
image = cv2.imread('/content/drive/MyDrive/datasets/people_counting_otus/96/001.jpg')

# Получение размера изображения
height, width, channels = image.shape
height, width, channels

(1080, 1920, 3)

In [ ]:
tracked_ids = None

def process_video(path):
  cap = cv2.VideoCapture(path)

  # Путь для сохранения видео
  output_video_path = f'output_{path}'
  # Создание объекта VideoWriter для записи видео
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))  # Замените на размер ваших кадров

  global tracked_ids
  tracked_ids = set()

  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break

      # Детекция людей
      detections = detect_people(frame)

      # Трекинг людей
      tracked_objects = track_people(detections, frame)

      # Подсчет уникальных людей
      count_unique_people(tracked_objects, path)
      total_people = len(tracked_ids)

      # Визуализация треков
      output_frame = visualize_tracks(tracked_objects, frame)

      # Отображение текущего подсчета людей
      cv2.putText(output_frame, f'Total People: {total_people}', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

      # Запись кадра в видеофайл
      out.write(frame)

  # Завершение записи видео
  cap.release()
  out.release()
  return output_video_path, total_people

In [ ]:
video_paths = ['video96.mp4', 'video1.mp4', 'video3.mp4']
output_videos = []

for vpath in video_paths:
  output_path, total_people = process_video(vpath)
  info_dict = {
      'output_path': output_path,
      'count': total_people
  }
  output_videos.append(info_dict)


0: 384x640 19 persons, 2 motorcycles, 454.5ms
Speed: 26.3ms preprocess, 454.5ms inference, 34.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 2 motorcycles, 314.3ms
Speed: 7.8ms preprocess, 314.3ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 persons, 3 motorcycles, 596.1ms
Speed: 8.1ms preprocess, 596.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 persons, 2 motorcycles, 281.2ms
Speed: 4.0ms preprocess, 281.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 2 motorcycles, 270.0ms
Speed: 4.2ms preprocess, 270.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 motorcycle, 1 horse, 275.8ms
Speed: 4.0ms preprocess, 275.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 1 motorcycle, 1 skateboard, 397.9ms
Speed: 7.7ms preprocess, 397.9ms inference, 1.8ms postproces